In [1]:
# reading Websites using Google API

#!pip install beautifulsoup4
#!pip install jupyter-tabnine
#!pip3 install jedi==0.17.2
#!pip install google

#!pip install html5lib
#!pip install bs4

%config Completer.use_jedi = False

In [15]:
try: 
    from googlesearch import search 
except ImportError:  
    print("No module named 'google' found") 

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
#nltk.download("all")
from nltk.tokenize import sent_tokenize, word_tokenize 

import re

import requests

from bs4 import BeautifulSoup

import datetime

import json

import pandas as pd

from pandas.io.json import json_normalize

In [3]:
#Google query for crime and getting all the returned urls

query = "crime"

results = []

for j in search(query, tld="com", num=1, stop=30, pause=2): 
    results.append(j)
    
#print(results)

['https://thetandd.com/news/local/crime-and-courts/bill-mandates-boater-ed-family-promotes-safety-after-fatal-crash/article_50577d8d-4870-5086-9b07-a649d511fdbd.html', 'https://richmond.com/news/local/crime-and-courts/chesterfield-man-charged-in-hit-and-run-crash-that-killed-henrico-police-captain/article_df4ea108-28ea-526c-80d8-d7ded5013781.html', 'https://en.wikipedia.org/wiki/Crime', 'https://www.mercurynews.com/tag/crime/', 'https://www.sjpd.org/records/crime-stats-maps', 'https://www.mercurynews.com/news/crime-public-safety/', 'https://www.neighborhoodscout.com/ca/san-jose/crime', 'https://www.foxnews.com/category/us/crime', 'https://people.com/crime/', 'https://www.fbi.gov/services/cjis/ucr', 'https://www.britannica.com/topic/crime-law', 'https://www.merriam-webster.com/dictionary/crime', 'https://www.pewresearch.org/fact-tank/2020/11/20/facts-about-crime-in-the-u-s/', 'https://crime-data-explorer.fr.cloud.gov/', 'https://www.cbsnews.com/crime/', 'https://www.unodc.org/unodc/en/o

In [4]:
#filtering crime urls

crime_urls = []
for r in results:
    crime_url = re.findall("https://[^<;&]+", r)
    #print("r is: ",r)
    crime_urls= crime_urls + crime_url
    #print("filtered URL is: ", crime_url)
    

filtered_crime_urls = [url for url in crime_urls
                     if 
                     'google' not in url and
                     'gstatic' not in url and
                      'tag' not in url and
                      'index' not in url]
#print(filtered_crime_urls)
    

['https://thetandd.com/news/local/crime-and-courts/bill-mandates-boater-ed-family-promotes-safety-after-fatal-crash/article_50577d8d-4870-5086-9b07-a649d511fdbd.html', 'https://richmond.com/news/local/crime-and-courts/chesterfield-man-charged-in-hit-and-run-crash-that-killed-henrico-police-captain/article_df4ea108-28ea-526c-80d8-d7ded5013781.html', 'https://en.wikipedia.org/wiki/Crime', 'https://www.sjpd.org/records/crime-stats-maps', 'https://www.mercurynews.com/news/crime-public-safety/', 'https://www.neighborhoodscout.com/ca/san-jose/crime', 'https://www.foxnews.com/category/us/crime', 'https://people.com/crime/', 'https://www.fbi.gov/services/cjis/ucr', 'https://www.britannica.com/topic/crime-law', 'https://www.merriam-webster.com/dictionary/crime', 'https://www.pewresearch.org/fact-tank/2020/11/20/facts-about-crime-in-the-u-s/', 'https://crime-data-explorer.fr.cloud.gov/', 'https://www.cbsnews.com/crime/', 'https://www.unodc.org/unodc/en/organized-crime/intro/emerging-crimes.html'

In [5]:
def get_domain_url (url):
    domain = str(url).split('/')
    domain_no_www = domain[2].replace("www.", "")
    #print(domain_no_www)
    return domain_no_www
    #print(domain[2])

In [6]:
def external_link_cnt(domain, links):
    cnt = 0
    for link in links:
        link_domain = get_domain_url(link)
        
        if link_domain.find(domain) !=-1:
            continue
        #print('\ndomain is:', domain, '\nlink_domain is:', link_domain)
        cnt= cnt+1
    return cnt

In [121]:
#Extracting sentences, urls, and modified date


#*******************placeholders****************************

#holds number of outgoing links. 
#I have filtered the links to the links that have a different domain name than the site being scraped
outgoing_links_count = 0

#List of JSON objects _ unflattened
final_output = []


#formatted modified date (if exists)
formatted_modified_date = ""



for url in filtered_crime_urls[:20]:
    
    #A list to hold the image urls in a website
    images = []
    
    #All sentences in the web content of a page
    sentences = []
    
    #filterd the sentences to those that have the queried keyword: crime
    crime_sentences = []
    
    #other temp variables
    tags = []
    links = []
    outgoing_links = []
    dates = []
    
    #print("\nurl is: ", url)
    r = requests.get(url, allow_redirects=True)
    soup = BeautifulSoup(r.content, 'lxml')
    ptags = soup.find_all('p')
    
    
    #getting sentence and filtering for keyword crime
    for p in ptags:
        txt = p.text
        sentences = sentences + sent_tokenize(txt)
    for sentence in sentences:
        if " " not in sentence:
            continue
        if "." not in sentence:
            continue
        if "crim" not in sentence.lower():
            continue
        if len(sentence)<2:
            continue
        #other str cleanups
        crime_sentences.append(str(sentence).replace(u'\xa0', u' ').replace('\n', '').replace('\r',''))
    #print(crime_sentences, "\n")
    if len(crime_sentences)==0:
        continue
    
    
    #getting images
    for img in soup.findAll('img'):
        #print(img.get('src'), "\n")
        images = images + re.findall("https://[^<;&]+", str(img.get('src')))
        
    #print("image urls in the web page: \n",images)
    
    #getting all the urls in the page
    tags = soup.find_all('a') 
    for tag in tags:
        links = links + re.findall("https://[^<;&]+", str(tag.get('href')))
    #finding the domain of the webpage to compare the urls in that page with and filter for only outgoing urls
    url_domain = get_domain_url(url)
    #outgoing urls should have different domain names
    outgoing_links_count = external_link_cnt(url_domain, links)
    
    
    #print(outgoing_links_count)
    #date modified
    #getting "dateModified":"YYYY_MM_DDTHH:MM:SS" and converting to time AM/PM mon DD YYYY 
    #some pages may not have this entry
    dates = re.findall(r'dateModified":"\d{4}\D{0,1}\d{1,2}\D{0,1}\d{1,2}T\d{2}:\d{2}:\d{2}', str(r.content))
    for date in dates:
        modified_date = date[15:]
        day = modified_date.split('T')[0]
        daystr = datetime.datetime.strptime(day, '%Y-%m-%d').strftime('%b %d %Y')
        time = modified_date.split('T')[1]
        timestr = datetime.datetime.strptime(time, '%H:%M:%S').strftime('%I:%M:%S %p')
        formatted_modified_date = timestr + " " + daystr
        #print(formatted_modified_date)
    
    
    #content is the dictionary created to contain Images, sentence, date_modified and #outgoing links
    content = {'Images':images, 'sentences': crime_sentences, 'num_outgoing_links': outgoing_links_count, 'website_last_modified': formatted_modified_date}
    #creating Json from the dictionary
    json_output = json.dumps(content)
    #Adding the JSON object to the list of JSON objects
    final_output.append(json_output)
    
#Converting JSON to dataframe just for practice

print("\n_____________________ Unflatten JSON List ______________________\n", final_output)


_____________________ Unflatten JSON List ______________________
 ['{"Images": ["https://bloximages.chicago2.vip.townnews.com/central.leetemplates.com/content/tncms/live/global/resources/images/logos/ds/400/thetandd.com.png?_dc=Mar.Mon.2021", "https://bloximages.chicago2.vip.townnews.com/central.leetemplates.com/content/tncms/live/global/resources/images/logos/ds/400/thetandd.com.png?_dc=Mar.Mon.2021", "https://bloximages.chicago2.vip.townnews.com/central.leetemplates.com/content/tncms/live/global/resources/images/logos/ds/400/thetandd.com.png?_dc=Mar.Mon.2021", "https://bloximages.chicago2.vip.townnews.com/thetandd.com/content/tncms/custom/image/9494352c-486e-11ea-b290-535df76c9f6c.png?resize=200%2C17", "https://bloximages.chicago2.vip.townnews.com/thetandd.com/content/tncms/assets/v3/editorial/2/9f/29f95bd7-be93-5d18-bd86-4ed0bccfed1b/603c1c75e1db2.image.jpg", "https://bloximages.chicago2.vip.townnews.com/thetandd.com/content/tncms/assets/v3/editorial/2/9f/29f95bd7-be93-5d18-bd86-4e

In [48]:
print (sent_col_cnt, img_col_cnt)

80 84


In [59]:
#Getting max number of columns for images and sentences

def max_number_of_sentences (json_data):
    max_col_sent = 0
    for entry in json_data:
        jdata = json.loads(entry)
        if len(jdata['sentences']) > max_col_sent:
            max_col_sent = len(jdata['sentences'])
    return max_col_sent



def max_number_of_images (json_data):
    max_col_img = 0
    for entry in json_data:
        jdata = json.loads(entry)
        if len(jdata['Images']) > max_col_img:
            max_col_img = len(jdata['Images'])
    return max_col_img

                

In [122]:
#Creating column headers for the flatten JSON entries.
#For sentences:

def create_sent_keys(max_number_of_sentences):
    sent_keys = []
    for i in range(max_number_of_sentences):
        sent_keys.append('Image_' +  str(i))
    return sent_keys


def create_img_keys (max_number_of_images):
    img_keys = []
    for i in range(max_number_of_images):
        img_keys.append('sentence_' + str(i))
    return(img_keys)

sent_headers = create_sent_keys(max_number_of_sentences(final_output))
col_headers = create_img_keys(max_number_of_images(final_output))

flatten_keys = sent_headers + col_headers 
flatten_keys.append('num_outgoing_links')
flatten_keys.append('website_last_modified')

In [123]:
csvdf = pd.DataFrame(columns = flatten_keys)
display(csvdf)

,Image_0,Image_1,Image_2,Image_3,Image_4,Image_5,Image_6,Image_7,Image_8,Image_9,...,sentence_76,sentence_77,sentence_78,sentence_79,sentence_80,sentence_81,sentence_82,sentence_83,num_outgoing_links,website_last_modified


In [115]:
# converting the unflatten JSON list to a flat Jason list

out = {}
flatten_json = []
row =0
for data in final_output:
    jdata = json.loads(data)
    #df_temp=pd.DataFrame.from_dict(pd.json_normalize(jdata), orient='columns')
    #frames.append(df_temp)
    i = 0
    for img in jdata['Images']:
        col = 'Image_'+str(i)
        out[col] = img
        i = i+1
    i = 0
    for sent in jdata['sentences']:
        col = 'sentence_' + str(i)
        out[col] = sent
        i = i + 1
    out['num_outgoing_links'] = jdata['num_outgoing_links']
    out['website_last_modified'] = jdata['website_last_modified']
    flatten_json.append(json.dumps(out))
    i = 0
    
flatten_json



['{"Image_0": "https://bloximages.chicago2.vip.townnews.com/central.leetemplates.com/content/tncms/live/global/resources/images/logos/ds/400/thetandd.com.png?_dc=Feb.Sun.2021", "Image_1": "https://bloximages.chicago2.vip.townnews.com/central.leetemplates.com/content/tncms/live/global/resources/images/logos/ds/400/thetandd.com.png?_dc=Feb.Sun.2021", "Image_2": "https://bloximages.chicago2.vip.townnews.com/central.leetemplates.com/content/tncms/live/global/resources/images/logos/ds/400/thetandd.com.png?_dc=Feb.Sun.2021", "Image_3": "https://bloximages.chicago2.vip.townnews.com/thetandd.com/content/tncms/custom/image/9494352c-486e-11ea-b290-535df76c9f6c.png?resize=200%2C17", "Image_4": "https://bloximages.chicago2.vip.townnews.com/thetandd.com/content/tncms/assets/v3/editorial/2/9f/29f95bd7-be93-5d18-bd86-4ed0bccfed1b/603c1c75e1db2.image.jpg", "Image_5": "https://bloximages.chicago2.vip.townnews.com/thetandd.com/content/tncms/assets/v3/editorial/2/9f/29f95bd7-be93-5d18-bd86-4ed0bccfed1b/6

In [124]:
#writing the JSON list into a dataframe for practice
#creating the csv output using the dataframe

frames = []
for data in flatten_json:
    jdata = json.loads(data)
    df_temp=pd.DataFrame.from_dict(pd.json_normalize(jdata), orient='columns')
    frames.append(df_temp)
csvout= pd.concat(frames, axis = 0)
display(csvout)

csvout.to_csv('flatten_content.csv',header=True, index = False)

,Image_0,Image_1,Image_2,Image_3,Image_4,Image_5,Image_6,sentence_0,num_outgoing_links,website_last_modified,...,Image_74,Image_75,Image_76,Image_77,Image_78,Image_79,Image_80,Image_81,Image_82,Image_83
0,https://bloximages.chicago2.vip.townnews.com/c...,https://bloximages.chicago2.vip.townnews.com/c...,https://bloximages.chicago2.vip.townnews.com/c...,https://bloximages.chicago2.vip.townnews.com/t...,https://bloximages.chicago2.vip.townnews.com/t...,https://bloximages.chicago2.vip.townnews.com/t...,https://bloximages.chicago2.vip.townnews.com/t...,Martha Rose Brown covers crime and other topics.,22,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://bloximages.chicago2.vip.townnews.com/c...,https://bloximages.chicago2.vip.townnews.com/c...,https://bloximages.chicago2.vip.townnews.com/c...,https://bloximages.chicago2.vip.townnews.com/t...,https://bloximages.chicago2.vip.townnews.com/t...,https://bloximages.chicago2.vip.townnews.com/t...,https://bloximages.chicago2.vip.townnews.com/t...,"In ordinary language, a crime is an unlawful a...",147,10:14:34 AM Feb 06 2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://bloximages.chicago2.vip.townnews.com/c...,https://bloximages.chicago2.vip.townnews.com/c...,https://bloximages.chicago2.vip.townnews.com/c...,https://bloximages.chicago2.vip.townnews.com/t...,https://bloximages.chicago2.vip.townnews.com/t...,https://bloximages.chicago2.vip.townnews.com/t...,https://bloximages.chicago2.vip.townnews.com/t...,The Uniform Crime Reporting (UCR) Program was ...,7,10:14:34 AM Feb 06 2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://production-nscout-static-lrsdmrzmrhqef...,https://production-nscout-static-lrsdmrzmrhqef...,https://production-nscout-static-lrsdmrzmrhqef...,https://production-nscout-static-lrsdmrzmrhqef...,https://production-nscout-static-lrsdmrzmrhqef...,https://production-nscout-static-lrsdmrzmrhqef...,https://production-nscout-static-lrsdmrzmrhqef...,Methodology: Our nationwide meta-analysis over...,5,10:14:34 AM Feb 06 2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://www.fbi.gov/++theme++fbigov.theme/imag...,https://www.fbi.gov/image-repository/ucr-logo....,https://www.fbi.gov/image-repository/nibrs-log...,https://www.fbi.gov/image-repository/hate-crim...,https://www.fbi.gov/image-repository/leoka-log...,https://www.fbi.gov/image-repository/use-of-fo...,https://www.fbi.gov/image-repository/crime-dat...,It also provides information for students of c...,14,10:14:34 AM Feb 06 2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://cdn.britannica.com/mendel/eb-logo/b-50...,https://cdn.britannica.com/safeimages/Presiden...,https://cdn.britannica.com/safeimages/presiden...,https://www.fbi.gov/image-repository/hate-crim...,https://www.fbi.gov/image-repository/leoka-log...,https://www.fbi.gov/image-repository/use-of-fo...,https://www.fbi.gov/image-repository/crime-dat...,"Crime, the intentional commission of an act u...",12,10:14:34 AM Feb 06 2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://merriam-webster.com/assets/mw/static/a...,https://merriam-webster.com/assets/mw/static/a...,https://merriam-webster.com/assets/mw/static/a...,https://merriam-webster.com/assets/mw/static/a...,https://www.facebook.com/tr?id=673022290083244,https://www.fbi.gov/image-repository/use-of-fo...,https://www.fbi.gov/image-repository/crime-dat...,"offense, sin, vice, crime, scandal mean a tran...",15,10:14:34 AM Feb 06 2021,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://www.pewresearch.org/wp-content/themes/...,https://www.pewresearch.org/wp-content/themes/...,https://www.pewresearch.org/wp-content/uploads...,https://www.pewresearch.org/wp-content/uploads...,https://www.pewresearch.org/wp-content/uploads...,https://www.pewresearch.org/wp-content/uploads...,https://www.pewresearch.org/wp-content/uploads...,From the first day of his presidency to his ca...,29,05:30:00 PM Nov 20 2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,https://cbsnews2.cbsistatic.com/hu